In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import gensim 
import sys
from parse import *
from prep_data import read_aact

In [ ]:
# Christine: gender; location; medication
# Charity: age; race; condition

### Read in data from AACT

In [2]:
data = read_aact()
df = pd.DataFrame(data)
df.columns = ['study_id', 'inclusion', 'exclusion', 'gender', 'age_min', 'age_max']
df

6795


,study_id,inclusion,exclusion,gender,age_min,age_max
0,NCT00000105,Inclusion Criteria:\n - Patients mus...,Exclusion Criteria:\n - Pregnant or ...,Both,18 Years,N/A
1,NCT00000106,Inclusion Criteria:\n - Patients are...,,Both,18 Years,65 Years
2,NCT00000107,Inclusion Criteria:\n - Resting bloo...,,Both,17 Years,60 Years
3,NCT00000108,Inclusion Criteria:\n - Postmenopaus...,,Female,50 Years,65 Years
4,NCT00000102,Inclusion Criteria:\n - diagnosed wi...,Exclusion Criteria:\n - history of l...,Both,14 Years,35 Years
5,NCT00000104,Inclusion Criteria:\n - Pregnant mot...,,Female,N/A,N/A
6,NCT00000117,"To be eligible, patients must have a history o...",,Both,N/A,50 Years
7,NCT00000118,All patients must have had AIDS as defined by ...,,Both,N/A,N/A
8,NCT00000119,Women and men 18 years or older with a documen...,Exclusion criteria include corneal pathology o...,Both,18 Years,N/A
9,NCT00000120,The study included male and female infants del...,,Both,N/A,1 Year


### Goal 1: 

Develop a query and a model for each primitive; output = {0,1} whether the inclusion criteria statement in question mentions (and, by extension, restricts on) the primitive in question. Note that a single clinical trial will generally include > 1 criteria statement. Run against MIMIC. Use either manual review and/or regex-based queries as ground truth to train model(s) on. 

#### Parse Race using existing tools

In [3]:
df['race'] = df['inclusion'].apply(lambda x: find_race(x))
df

,study_id,inclusion,exclusion,gender,age_min,age_max,race
0,NCT00000105,Inclusion Criteria:\n - Patients mus...,Exclusion Criteria:\n - Pregnant or ...,Both,18 Years,N/A,
1,NCT00000106,Inclusion Criteria:\n - Patients are...,,Both,18 Years,65 Years,
2,NCT00000107,Inclusion Criteria:\n - Resting bloo...,,Both,17 Years,60 Years,
3,NCT00000108,Inclusion Criteria:\n - Postmenopaus...,,Female,50 Years,65 Years,
4,NCT00000102,Inclusion Criteria:\n - diagnosed wi...,Exclusion Criteria:\n - history of l...,Both,14 Years,35 Years,
5,NCT00000104,Inclusion Criteria:\n - Pregnant mot...,,Female,N/A,N/A,
6,NCT00000117,"To be eligible, patients must have a history o...",,Both,N/A,50 Years,
7,NCT00000118,All patients must have had AIDS as defined by ...,,Both,N/A,N/A,
8,NCT00000119,Women and men 18 years or older with a documen...,Exclusion criteria include corneal pathology o...,Both,18 Years,N/A,
9,NCT00000120,The study included male and female infants del...,,Both,N/A,1 Year,
